In [1]:
import os
import folium
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Polygon

from utils import *

### Fountain

In [2]:
fountains = pd.read_csv(os.getenv("fountains"))
fountains.columns = fountains.columns.str.lower()

In [3]:
fountains = gpd.GeoDataFrame(fountains, geometry=gpd.points_from_xy(fountains.longitude, fountains.latitude))

In [4]:
print(fountains.crs)

None


In [5]:
fountains = fountains.set_crs('epsg:4326')

In [6]:
print(fountains.crs)

epsg:4326


### calcul the distance between fountain and closest one

In [7]:
# fountains_utm = fountains.to_crs(epsg=32188)

In [8]:
from scipy.spatial import cKDTree

# Create a KDTree from the geometry of the GeoDataFrame
tree = cKDTree(np.array(fountains.geometry.apply(lambda geom: [geom.x, geom.y])).tolist())

# Query the tree for the closest points to each point in the GeoDataFrame
distances, indices = tree.query(np.array(fountains.geometry.apply(lambda geom: [geom.x, geom.y])).tolist(), k=2)

# Get the closest point for each row
fountains['closest_point'] = fountains.geometry.iloc[indices[:, 1]].values

In [9]:
from geopy.distance import distance

def get_distance(row):
    point = row['geometry']
    closest_point = row['closest_point']
    return distance((point.y, point.x), (closest_point.y, closest_point.x)).meters

In [10]:
fountains["closest_fountain"] = fountains.apply(get_distance, axis=1)

In [11]:
m = folium.Map(location=[45.5414195, -73.6303031], tiles="cartodbpositron", zoom_start=12)

In [12]:
part = fountains[fountains.nom_parc_lieu.str.contains("Royal", na=False)]
for _, row in part.iterrows():
    lat = row.latitude
    lon = row.longitude
    folium.Marker(location=[lat, lon], popup=row[["id", "longitude", "latitude"]]).add_to(m)

In [13]:
m

In [14]:
# https://stackoverflow.com/questions/19412462/getting-distance-between-two-points-based-on-latitude-longitude

from geopy.distance import geodesic


origin = (45.504592, -73.582003)  # (latitude, longitude) don't confuse
dist = (45.505103, -73.583063)

print(geodesic(origin, dist).meters)  # 23576.805481751613
print(geodesic(origin, dist).kilometers)  # 23.576805481751613
print(geodesic(origin, dist).miles)  # 14.64994773134371

100.43916649131438
0.10043916649131439
0.0624100046300321


In [15]:
df = pd.DataFrame({
    'id': [1, 2, 3, 4],
    'long': [-73.582003, -73.587419, -73.586842, -73.587095],
    'lat': [45.504592, 45.503420, 45.503720, 45.502097]})

df

,id,long,lat
0,1,-73.582003,45.504592
1,2,-73.587419,45.503420
2,3,-73.586842,45.503720
3,4,-73.587095,45.502097


In [16]:
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.long, df.lat))

In [17]:
print(gdf.crs)

None


In [18]:
gdf = gdf.set_crs('epsg:32188')

In [19]:
print(gdf.crs)

epsg:32188


In [20]:
gdf.sjoin_nearest(gdf.copy(), distance_col="distance")

,id_left,long_left,lat_left,geometry,index_right,id_right,long_right,lat_right,distance
0,1,-73.582003,45.504592,POINT (-73.582 45.505),0,1,-73.582003,45.504592,0.0
1,2,-73.587419,45.503420,POINT (-73.587 45.503),1,2,-73.587419,45.503420,0.0
2,3,-73.586842,45.503720,POINT (-73.587 45.504),2,3,-73.586842,45.503720,0.0
3,4,-73.587095,45.502097,POINT (-73.587 45.502),3,4,-73.587095,45.502097,0.0


In [21]:
m = folium.Map(location=[45.5414195, -73.6303031], tiles="cartodbpositron", zoom_start=12)

In [22]:
for _, row in gdf.iterrows():
    lat = row.lat
    lon = row.long
    folium.Marker(location=[lat, lon], popup=row["id"]).add_to(m)

In [23]:
m

In [24]:
gdf.sjoin_nearest(gdf.copy(), distance_col="nearest")

,id_left,long_left,lat_left,geometry,index_right,id_right,long_right,lat_right,nearest
0,1,-73.582003,45.504592,POINT (-73.582 45.505),0,1,-73.582003,45.504592,0.0
1,2,-73.587419,45.503420,POINT (-73.587 45.503),1,2,-73.587419,45.503420,0.0
2,3,-73.586842,45.503720,POINT (-73.587 45.504),2,3,-73.586842,45.503720,0.0
3,4,-73.587095,45.502097,POINT (-73.587 45.502),3,4,-73.587095,45.502097,0.0


In [25]:
# import numpy as np
# from scipy.spatial import KDTree

# df = pd.DataFrame({
#     'id': [1, 2, 3, 4],
#     'long': [-73.582003, -73.587419, -73.586842, -73.587095],
#     'lat': [45.504592, 45.503420, 45.503720, 45.502097]})

# gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.long, df.lat))
# gdf = gdf.set_crs('epsg:32188')

# # Create a KDTree from the points in the GeoDataFrame
# tree = KDTree(gdf.geometry.values.data)

# # Query the tree to find the distance from each point to its nearest neighbor
# distances, indices = tree.query(gdf.geometry.values.data, k=2)

# # Create a new DataFrame with the distances to the nearest neighbor
# result = pd.DataFrame({
#     'id_left': gdf['id'],
#     'distance': distances[:, 1]
# })

# print(result)


In [26]:
from scipy.spatial import KDTree

# Extract the coordinates of each point in the GeoDataFrame
coords = gdf.geometry.apply(lambda p: [p.x, p.y]).tolist()

# Create a NumPy array from the coordinates
data = np.array(coords)

# Create a KDTree from the NumPy array
tree = KDTree(data)

# Query the tree to find the distance from each point to its nearest neighbor
distances, indices = tree.query(data, k=2)

# Extract the distance to the second nearest neighbor (the first nearest neighbor is always the point itself)
distances = distances[:, 1]

# Add the distances to the original GeoDataFrame
gdf['distance'] = distances


In [27]:
gdf

,id,long,lat,geometry,distance
0,1,-73.582003,45.504592,POINT (-73.582 45.505),0.004917
1,2,-73.587419,45.503420,POINT (-73.587 45.503),0.000650
2,3,-73.586842,45.503720,POINT (-73.587 45.504),0.000650
3,4,-73.587095,45.502097,POINT (-73.587 45.502),0.001362
